In [1]:
from htmd.ui import *
from htmd.home import home
from os.path import join
config(viewer='ngl')
datadir = home(dataDir='building-protein-ligand')


Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

You are on the latest HTMD version (unpackaged : /shared/sdoerr/Work/htmdacellera/htmd).

Deprecation warning: "from htmd import *" will be deprecated. 
To import all HTMD shortcuts please from now on use "from htmd.ui import *"


# System building: trypsin-benzamidine

(Download the required files for the tutorial from this [link](http://pub.htmd.org/73hboiwia98hdj209jq0/bentryp.tar.gz).)


First we will download the crystal structure for 3PTB (Trypsin bound with Benzamidine) from the RCSB servers.

In [2]:
# We can use the pdb file found in the HTMD data directory
prot = Molecule(join(datadir, 'trypsin.pdb'))
# Or we can download it directly from the RCSB servers
prot = Molecule('3PTB')

2017-07-12 12:30:40,113 - htmd.molecule.readers - INFO - Using local copy for 3PTB: /shared/sdoerr/Work/htmdacellera/htmd/data/pdb/3ptb.pdb


In [3]:
prot.view()

The installed widget Javascript is the wrong version.


## Clean the structures

The PDB crystal structure contains the protein as well as water molecules, a calcium ion and a ligand. Here we will start by removing the ligand from the protein Molecule as we will add it later to manipulate it separately.

In [4]:
prot.remove('resname BEN')

2017-07-12 12:30:40,576 - htmd.molecule.molecule - INFO - Removed 9 atoms. 1692 atoms remaining in the molecule.


array([1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638], dtype=int32)

## Preparing the protein

In [5]:
prot = proteinPrepare(prot)

2017-07-12 12:30:40,749 - propka - INFO - No pdbfile provided
2017-07-12 12:30:43,160 - htmd.builder.preparation - WARNING - The following residue has not been optimized: CA
2017-07-12 12:30:49,013 - htmd.builder.preparationdata - INFO - The following residues are in a non-standard state: CYS    22  A (CYX), HIS    40  A (HIE), CYS    42  A (CYX), HIS    57  A (HIP), CYS    58  A (CYX), HIS    91  A (HID), CYS   128  A (CYX), CYS   136  A (CYX), CYS   157  A (CYX), CYS   168  A (CYX), CYS   182  A (CYX), CYS   191  A (CYX), CYS   201  A (CYX), CYS   220  A (CYX), CYS   232  A (CYX)
2017-07-12 12:30:49,014 - htmd.builder.preparationdata - WARNING - Dubious protonation state: the pKa of 4 residues is within 1.0 units of pH 7.0.
2017-07-12 12:30:49,016 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS    57  A (pKa= 7.46)
2017-07-12 12:30:49,017 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    GLU    70  A (pKa= 6.10)
2017-07-12 12:30:49,

## Define segments

To build a system in HTMD, we need to separate the chemical molecules into separate segments. This prevents the builder from accidentally bonding different chemical molecules and allows us to add caps to them.

In [6]:
prot = autoSegment(prot, sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')

2017-07-12 12:30:49,479 - htmd.builder.builder - INFO - Created segment P0 between resid 16 and 245.


Center the protein to the origin

In [7]:
prot.center()

## Let's work on the ligand!

In [8]:
ligand = Molecule(join(datadir, 'benzamidine.mol2'))
ligand.center()
ligand.view()

The installed widget Javascript is the wrong version.


In [9]:
# We can give a convenient segid and resname to the ligand
# The resname should be MOL to match the parameters in the
# rtf and prm files.
ligand.set('segid','L')
ligand.set('resname','MOL')

But the ligand is located inside the protein...
We would like the ligand to be:

* At a certain distance from the protein
* Rotated randomly, to provide different starting conditions

## Let's randomize the ligand position

In [10]:
ligand.rotateBy(uniformRandomRotation())

This took care of the ligand rotation around its own center. 
We still need to position it far from the protein.
First, find out the radius of the protein:

![](http://pub.htmd.org/73hboiwia98hdj209jq0/maxdist.png)

In [11]:
from htmd.molecule.util import maxDistance
D = maxDistance(prot, 'all')
print(D)

28.832803412


In [12]:
D += 10
# Move the ligand 10 Angstrom away from the furthest protein atom in X dimension
ligand.moveBy([D, 0, 0])  
# rotateBy rotates by default around [0, 0, 0]. Since the ligand has been moved
# away from the center it will be rotated in a sphere of radius D+10 around [0, 0, 0]
ligand.rotateBy(uniformRandomRotation())

### Mix it all together

In [14]:
mol = Molecule(name='combo')
mol.append(prot)
mol.append(ligand)
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='resname MOL', style='Licorice')
mol.view()

The installed widget Javascript is the wrong version.


## Solvate

> Water is the driving force of all nature. --Leonardo da Vinci

![](http://pub.htmd.org/73hboiwia98hdj209jq0/waterbox.png)

In [15]:
# We solvate with a larger box to fully solvate the ligand
DW = D + 5
smol = solvate(mol, minmax=[[-DW, -DW, -DW], [DW, DW, DW]])
smol.reps.add(sel='water', style='Lines')
smol.view()

2017-07-12 12:31:34,135 - htmd.builder.solvate - INFO - Using water pdb file at: /shared/sdoerr/Work/htmdacellera/htmd/builder/wat.pdb
2017-07-12 12:31:35,207 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2


Solvating: 100% (8/8) [############################################] eta 00:00 /


2017-07-12 12:31:43,551 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 20149 water molecules were added to the system.
The installed widget Javascript is the wrong version.


## Available CHARMM forcefields

In [16]:
charmm.listFiles()

---- Topologies files list: /shared/sdoerr/Work/htmdacellera/htmd/builder/charmmfiles/top/ ----
top/top_all22_prot.rtf
top/top_all22star_prot.rtf
top/top_all35_ethers.rtf
top/top_all36_carb.rtf
top/top_all36_cgenff.rtf
top/top_all36_lipid.rtf
top/top_all36_na.rtf
top/top_all36_prot.rtf
top/top_water_ions.rtf
---- Parameters files list: /shared/sdoerr/Work/htmdacellera/htmd/builder/charmmfiles/par/ ----
par/par_all22_prot.prm
par/par_all22star_prot.prm
par/par_all35_ethers.prm
par/par_all36_carb.prm
par/par_all36_cgenff.prm
par/par_all36_lipid.prm
par/par_all36_na.prm
par/par_all36_prot.prm
par/par_all36_prot_mod.prm
par/par_water_ions.prm
---- Stream files list: /shared/sdoerr/Work/htmdacellera/htmd/builder/charmmfiles/str/ ----
str/carb/toppar_all36_carb_glycolipid.str
str/carb/toppar_all36_carb_glycopeptide.str
str/carb/toppar_all36_carb_model.str
str/lipid/toppar_all36_lipid_bacterial.str
str/lipid/toppar_all36_lipid_cardiolipin.str
str/lipid/toppar_all36_lipid_cholesterol.str
str/l

## Build and ionize using CHARMM

In [17]:
topos  = charmm.defaultTopo() + [join(datadir, 'benzamidine.rtf')]
params = charmm.defaultParam() + [join(datadir, 'benzamidine.prm')]

bmol = charmm.build(smol, topo=topos, param=params, 
                        outdir='./build')

2017-07-12 12:31:54,834 - htmd.builder.charmm - INFO - Writing out segments.
2017-07-12 12:32:04,975 - htmd.builder.builder - INFO - 6 disulfide bonds were added


Bond between A: [serial 351 resid 42 resname CYX chain A segid P0]
             B: [serial 571 resid 58 resname CYX chain A segid P0]

Bond between A: [serial 1683 resid 136 resname CYX chain A segid P0]
             B: [serial 2611 resid 201 resname CYX chain A segid P0]

Bond between A: [serial 2146 resid 168 resname CYX chain A segid P0]
             B: [serial 2353 resid 182 resname CYX chain A segid P0]

Bond between A: [serial 1604 resid 128 resname CYX chain A segid P0]
             B: [serial 3004 resid 232 resname CYX chain A segid P0]

Bond between A: [serial 2494 resid 191 resname CYX chain A segid P0]
             B: [serial 2799 resid 220 resname CYX chain A segid P0]

Bond between A: [serial 92 resid 22 resname CYX chain A segid P0]
             B: [serial 1988 resid 157 resname CYX chain A segid P0]



2017-07-12 12:32:05,377 - htmd.builder.charmm - INFO - Starting the build.
2017-07-12 12:32:06,096 - htmd.builder.charmm - WARNING - Failed to set coordinates for 8 atoms.
2017-07-12 12:32:06,097 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 3 atoms due to bad angles.
2017-07-12 12:32:06,098 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 15 atoms.
2017-07-12 12:32:06,099 - htmd.builder.charmm - WARNING - Please check /shared/sdoerr/Work/htmdacellera/tutorials/build/log.txt for further information.
2017-07-12 12:32:06,099 - htmd.builder.charmm - INFO - Finished building.
2017-07-12 12:32:09,191 - htmd.builder.ionize - INFO - Adding 10 anions + 0 cations for neutralizing and 0 ions for the given salt concentration.
2017-07-12 12:32:09,615 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2017-07-12 12:32:09,616 - htmd.builder.ionize - INFO - Min distance between ions: 5A
2017-07-12 12:32:09,617 - htmd.builder.ionize - INFO - Placi

## Visualize

In [18]:
bmol.view()

The installed widget Javascript is the wrong version.
